# Homework 3: List Comprehensions, Exceptions, File I/O, and Recursion

## Question 1
Write a function called `scalar_multiply` which takes 2 arguments -- an `int` and a sequence (e.g. a `list`) of `int`s, of any length.

The function should return a new `list` where each element in the sequence has been multiplied by the `int`.

The function should raise a `ValueError` if the sequence given is empty, with any message provided.

The function should otherwise always return a `list`, even if the provided sequence is not a list.

Do not use a `for`-loop or a `while`-loop, you must use a list comprehension. **No points will be awarded if `for`-/`while`-loops are used.**

As an example, `scalar_multiply(3, [2, 4, 5])` should return the list` [6, 12, 15]`, where each element has been multiplied by `3`.

Calling `scalar_multiply(11, [])` should raise a `ValueError`.

[2 points]

In [1]:
def scalar_multiply(scalar, sequence):
    if not sequence:
        raise ValueError("The sequence is empty!")
    return [scalar * element for element in sequence]

In [2]:
assert scalar_multiply(3, [2, 4, 5]) == [6, 12, 15]

In [3]:
# autograder tests
try:
    scalar_multiply(2, [])
except ValueError:
    pass
else:
    raise ValueError("Should have failed!")

# test to see if a list comprehension was used
import ast, inspect

uses_list_comp = False
func = ast.parse(inspect.getsource(scalar_multiply))
for node in ast.walk(func):
    if type(node) == ast.ListComp:
        uses_list_comp = True

assert uses_list_comp, "No list comprehension used"

## Question 2

_Preamble: In class, we worked with text files and learned how to read and write files. Be sure to review the lecture notes. When working with numerical data, you will often encounter data in the form of [CSV files](https://en.wikipedia.org/wiki/Comma-separated_values). Take a moment to familiarize yourself with the [`csv` module](https://docs.python.org/3/library/csv.html) from Python's standard library, which provides an easy way to read & write CSV-based data._

Below is a function called `download_park_data` that downloads data from New York State about State Park annual attendence since 2003, and returns it as a string. You can take a look at the data directly through their [website](https://data.ny.gov/Recreation/State-Park-Annual-Attendance-Figures-by-Facility-B/8f3n-xj78/data).

Implement a function called `get_palisades_attendance` that does not take in any arguments. This function should do the following:

1. **Get data**: Call the `download_park_data` function.
2. **Write data to file**: Save the data that the `download_park_data` function returns by writing it to a file on your computer with a context manager. Name the written file `"downloaded_data.csv"`. It should not be saved in any particular directory. That is, use `open("downloaded_data.csv", "w")` with no file path. You do not need to use the `csv` module here.
3. **Read the file**: Then, use another context manager to read the lines of the file you just saved. You should be left with a `list` of `str`s.  You do not need to use the `csv` module here.
4. **Parse the file**: After you've read the lines of the file, use the `csv` module to parse the lines of data into a single `list` containing multiple of `list`s that represents rows of data.
5. **Calculate result**: Calculate the total attendance (`total_attendance`) from 2003 to 2020 (inclusive) for the `"Appalachian Trail"` in the `"Palisades"` OPRHP region. Return the value. The total attendence should be 156,506 people.

Remember, you may **not** use additional imports for your implementation (you can import what you want in order to test your implementation though, if you wish).

[2 points]

In [4]:
import csv
import requests

def download_park_data():
    response = requests.get(
        "https://data.ny.gov/api/views/8f3n-xj78/rows.csv?accessType=DOWNLOAD&sorting=true"
    )
    data = response.text
    return data


def get_palisades_attendance():
    data = download_park_data()
    with open("downloaded_data.csv", "w") as f:
        f.write(data)
    with open("downloaded_data.csv") as f:
        file_data = f.readlines()
        reader = csv.reader(file_data, delimiter=",")
    
    rows = [r for r in reader]
    total_attendance = 0
    for row in rows[1:]:  # skip the first row of Headers
        if (
            2003 <= int(row[0]) <= 2020 
            and row[1] == "Palisades" 
            and row[3] == "Appalachian Trail"
        ):
            total_attendance += int(row[4])
    return total_attendance


In [5]:
# autograder tests
# delete any downloaded csv data before starting for a clean slate
from pathlib import Path
f = Path("downloaded_data1.csv")
try:
    f.unlink()  # deletes
except Exception as e:
    if isinstance(e, FileNotFoundError):
        pass
    else:
        print("Couldn't delete old file - may be left in an unclean state.")

assert 156506 == get_palisades_attendance()

In [6]:
# ensure a context manager has been used
has_context_manager = False
func = ast.parse(inspect.getsource(get_palisades_attendance))
for node in ast.walk(func):
    if type(node) == ast.With:
        has_context_manager = True

assert has_context_manager

## Question 3

Write a recursive function `sum_list` to find the sum of all elements in a list. You can assume the list is flat. You **can not** use the built-in function `sum()`. You **can not** use iteration via a `for` or `while` loop. Your function must return an integer. You can assume you are passed a list of real integers. If your solution uses exceptions, the except statements must not be bare (i.e. the must specify the errors they can catch). You must handle the case where the given list is empty; the sum would be zero in this case.

[2 points]

In [7]:
def sum_list(a_list):
    # handle some trival list cases
    if len(a_list) < 1:
        return 0
    if len(a_list) == 1:
        return a_list[0]

    # list must be >= 2
    if len(a_list) == 2:
        a, b = a_list
        return a + b

    else:
        # First element
        a = a_list[0]
        # All the rest
        b = sum_list(a_list[1:])
        return a + b

In [8]:
# autograder tests
assert sum_list([]) == 0
for i in range(1, 100):
    list_ = list(range(i))
    assert sum_list(list_) == sum(list_)

In [9]:
# autograder tests
from bdb import Bdb
import sys

class RecursionDetected(Exception):
    pass

class RecursionDetector(Bdb):
    def do_clear(self, arg):
        pass

    def __init__(self, *args):
        Bdb.__init__(self, *args)
        self.stack = set()

    def user_call(self, frame, argument_list):
        code = frame.f_code
        if code in self.stack:
            raise RecursionDetected
        self.stack.add(code)

    def user_return(self, frame, return_value):
        self.stack.remove(frame.f_code)

def test_recursion(func):
    detector = RecursionDetector()
    detector.set_trace()
    try:
        func()
    except RecursionDetected:
        return True
    else:
        return False
    finally:
        sys.settrace(None)
        
assert test_recursion(lambda: sum_list(list(range(10))))

## Question 4
Create a recursive function `fib` to find the nth [Fibonacci number](https://en.wikipedia.org/wiki/Fibonacci_number). Assume that the the sequence starts with `0, 1, 1, ...`.

```py
>>> fib(0)
0
>>> fib(5)
5
>>> fib(10)
55
```

If your solution uses exceptions, the except statements must not be bare (ie. the must specify the errors they can catch).

[2 points]

In [10]:
def fib(n):
    if n < 2:
        return n
    return fib(n - 2) + fib(n - 1)

In [11]:
# autograder tests
assert fib(0) == 0
assert fib(5) == 5
assert fib(10) == 55

In [12]:
# autograder tests
assert fib(20) == 6765
assert test_recursion(lambda: fib(5))

## Question 5

Write a function `verify` that takes a string and returns the string if it contains at least one number, one uppercase letter, and one lowercase letter. The function should raise a `ValueError` otherwise. The `ValueError` should have the exact message `"Must contain a number, an uppercase letter, and a lowercase letter"`.

[1 point]

In [13]:
def verify(string):
    has_upper, has_lower, has_digit = False, False, False
    for char in string:
        if char.isupper():
            has_upper = True
        elif char.islower():
            has_lower = True
        elif char in "0123456789":
            has_digit = True
    if all((has_upper, has_lower, has_digit)):
        return string
    raise ValueError("Must contain a number, an uppercase letter, and a lowercase letter")

In [14]:
# autograder tests
msg = 'Must contain a number, an uppercase letter, and a lowercase letter'

try:
    verify('ab1')
    assert False, 'Should not pass'
except ValueError as e:
    assert str(e) == msg
    
try:
    verify('AB1')
    assert False, 'Should not pass'
except ValueError as e:
    assert str(e) == msg
    
try:
    verify('Abc')
    assert False, 'Should not pass'
except ValueError as e:
    assert str(e) == msg
    
try:
    verify('Ab1')
    assert True
except ValueError as e:
    assert False, 'Should pass'


## Question 6

Implement a function called `is_ten` that takes in one argument, an `int`, and returns `True` if the argument is equal to `10`, and `False` if not.

Then, implement a second function called `test_is_ten` that takes in no arguments, and calls the first function, `is_ten` at least twice. For each time you call `is_ten`, use `assert` statements to test the expected return value equals actual return value.

[1 point]

In [16]:
def is_ten(x):
    return x == 10
    
def test_is_ten():
    assert is_ten(10)
    assert is_ten(11) is False

In [17]:
# autograder tests
test_is_ten()  # should just pass

In [18]:
# autograder tests
# check the code itself to see if there are at least two assert statements
num_of_asserts = 0
func = ast.parse(inspect.getsource(test_is_ten))
for node in ast.walk(func):
    if type(node) == ast.Assert:
        num_of_asserts += 1
        
assert num_of_asserts >= 2